In [1]:
!pwd
!export PYTHONPATH=$PYTHONPATH:$(pwd)/evidential-learning-pytorch

import sys
print(sys.path)
sys.path.append('/work/DRO-EDL/1d/evidential-learning-pytorch')

from tqdm import tqdm
from scipy.stats import norm

import numpy as np
from scipy.stats import norm, invgamma

import matplotlib.pyplot as plt
from scipy.special import gamma
from scipy.optimize import minimize
from scipy.integrate import dblquad

import torch

from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from edl_pytorch import NormalInvGamma, evidential_regression

import torch.optim as optim

from scipy.stats import invgamma, norm

/work/DRO-EDL/2d
['/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/opencda/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [2]:
def cart2polar(x,y):
    r = (x**2 + y**2)**(1/2)
    theta = np.arctan2(y,x)
    return r , theta

def polar2cart(r,theta):
    x = r*np.cos(theta)
    y = r*np.sin(theta)
    return x,y

In [3]:
torch.manual_seed(0)

model = nn.Sequential(
    nn.Linear(2, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    NormalInvGamma(64, 2),
)
model.load_state_dict(torch.load(f'uncertain_weights/{90}.pth', weights_only=True))


<All keys matched successfully>

In [4]:
def NIG_sample(params):
    mu_0, lambda_, alpha, beta = params
    x_sigma2_dist = invgamma(alpha, scale=beta)
    sigma2 = x_sigma2_dist.rvs(1)
    x_mu_dist = norm(mu_0, np.sqrt(sigma2 / lambda_))
    mu = x_mu_dist.rvs(1)
    return np.array([mu[0], sigma2[0]])

In [5]:
import pickle
a_memory = dict()
with open(f'../1d/a_memory_set/a_memory_{0.9}.pickle', 'rb') as f:
    a_memory = pickle.load(f)

In [6]:
from matplotlib.patches import Polygon
from ipywidgets import interact, FloatSlider

def plot(x, y):
    fig, ax = plt.subplots(figsize=(8,8))
    
    print(x, y)
    sample = [x, y]
    input_data = torch.tensor(cart2polar(*sample), dtype=torch.float)[None,:]
    with torch.no_grad():
        pred = model(input_data)
    x_params = [pred[0][0,0], pred[1][0,0], pred[2][0,0], pred[3][0,0]]
    y_params = [pred[0][0,1], pred[1][0,1], pred[2][0,1], pred[3][0,1]]

    
    # ambiguity set
    ## x axis
    mu_0, lambda_, alpha, beta = x_params
    try:
        zx, zy = a_memory[float(f'{alpha:.02f}')]
    except:
        zx, zy = a_memory[float(f'{1.01}')]
    delta = zx / np.sqrt(lambda_/beta)
    x_mu_low, x_mu_high = mu_0 - delta, mu_0 + delta
    
    ## y axis
    mu_0, lambda_, alpha, beta = y_params
    try:
        zx, zy = a_memory[float(f'{alpha:.02f}')]
    except:
        zx, zy = a_memory[float(f'{1.01}')]
    delta = zx / np.sqrt(lambda_/beta)
    y_mu_low, y_mu_high = mu_0 - delta, mu_0 + delta
    
    vertices = [(x_mu_low, y_mu_low), (x_mu_low, y_mu_high), (x_mu_high, y_mu_high), (x_mu_high, y_mu_low)]
    
    rectangle = Polygon(vertices, closed=True, color='red')
    # sampling
    for _ in range(500):
        x_dist_params = NIG_sample(x_params)
        y_dist_params = NIG_sample(y_params)

        mean = np.array([x_dist_params[0], y_dist_params[0]])
        cov = np.array([
            [x_dist_params[1], 0],
            [0, y_dist_params[1]]
        ])


        # 공분산 행렬의 고유값과 고유벡터 계산
        eigvals, eigvecs = np.linalg.eigh(cov)

        # 1-sigma 수준에서 타원의 축 반지름 계산
        axis_lengths = np.sqrt(eigvals)

        # 타원의 각도 계산 (라디안)
        angle = np.arctan2(eigvecs[1, 0], eigvecs[0, 0])

        # 타원 좌표 생성
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse = np.array([axis_lengths[0] * np.cos(theta), axis_lengths[1] * np.sin(theta)])

        # 타원의 회전 적용
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                                     [np.sin(angle), np.cos(angle)]])
        rotated_ellipse = rotation_matrix @ ellipse

        # 타원을 평균 좌표로 이동
        ellipse_x, ellipse_y = rotated_ellipse[0] + mean[0], rotated_ellipse[1] + mean[1]
        if (x_mu_low < mean[0] and mean[0] < x_mu_high) and (y_mu_low < mean[1] and mean[1] < y_mu_high):
            color = 'lime'
            alpha = 0.1
            label = 'in ambiguity set'
        else:
            color = 'skyblue'
            alpha = 0.8
            label = 'out of ambiguity set'
        plt.plot(ellipse_x, ellipse_y, color=color, alpha=alpha, label=label)
    ax.add_patch(rectangle)

    plt.scatter(*sample, color='g', label='Ground Truth')
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    handles, labels = plt.gca().get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))  # 중복 제거

    # 고유한 레이블만 포함하는 legend 표시
    plt.legend(unique_labels.values(), unique_labels.keys())   
    plt.show()


interact(
        plot,
        x=FloatSlider(min=-100, max=100, step=1, value=0, description='Major Axis (a)'),
        y=FloatSlider(min=-100, max=100, step=1, value=0, description='Minor Axis (b)'),
);


interactive(children=(FloatSlider(value=0.0, description='Major Axis (a)', min=-100.0, step=1.0), FloatSlider(…

In [122]:
def plot_ellipse(mean, covariance, n_std=2, ax=None, **kwargs):
    """
    평균과 공분산 행렬을 사용하여 타원을 그립니다.

    Parameters:
    - mean: 1x2 배열, 타원의 중심 (평균 벡터)
    - covariance: 2x2 배열, 공분산 행렬
    - n_std: float, 타원의 크기를 결정하는 표준편차의 배수 (기본값: 2)
    - ax: matplotlib 축 객체 (기본값: None)
    - kwargs: matplotlib.patches.Ellipse에 전달할 추가 스타일 인수

    Returns:
    - 타원을 그린 matplotlib Ellipse 객체
    """
    from matplotlib.patches import Ellipse
    if ax is None:
        ax = plt.gca()

    # 공분산 행렬의 고유값과 고유벡터 계산
    eigenvalues, eigenvectors = np.linalg.eigh(covariance)

    # 고유값의 제곱근으로 타원의 축 길이 결정
    axis_length = n_std * np.sqrt(eigenvalues)

    # 고유벡터에서 타원의 회전 각도 계산
    angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))

    # 타원 생성
    ellipse = Ellipse(
        xy=mean,
        width=2 * axis_length[0],
        height=2 * axis_length[1],
        angle=angle,
        **kwargs
    )

    # 타원을 플롯에 추가
    ax.add_patch(ellipse)

    return ellipse

In [223]:
point_x, point_y = 7, 2
N = 2
mu_x = torch.linspace(1, 4, N)
mu_y = torch.linspace(6, 10, N)


vertices = [(mu_x.min(), mu_y.min()), (mu_x.min(), mu_y.max()), (mu_x.max(), mu_y.max()), (mu_x.max(), mu_y.min())]

rectangle = Polygon(vertices, closed=True, color='red')

sigma2_x = torch.linspace(1.1, 1.7, N)
sigma2_y = torch.linspace(1.6, 2.4, N)

mu_x_grid, mu_y_grid = torch.meshgrid(mu_x, mu_y, indexing="ij")
# means = torch.stack([mu_x_grid.flatten(), mu_y_grid.flatten()]).T
means = torch.tensor([1, 6])[None,:]


sigma2_x_grid, sigma2_y_grid = torch.meshgrid(sigma2_x, sigma2_y, indexing="ij")

pairs = torch.stack([sigma2_x_grid.flatten(), sigma2_y_grid.flatten()])
z = torch.zeros_like(pairs[0,:])
# cov = torch.stack([pairs[0,:], z, z, pairs[1,:]], dim=0).reshape(2,2,-1).permute(2,0,1)
cov = torch.tensor([[1.1, 0], [0, 1.6]])[None,:]

print("COIV")
print(cov)
chol_matrices = torch.linalg.cholesky(cov)
print("L")
print(chol_matrices)
# cholesky_matrix.shape


# Unscented transformation parameters
alpha = 1e-3  # Scale factor
beta = 2.0    # Optimal for Gaussian distributions
kappa = 0.0   # Secondary scaling parameter
n = means.shape[1]
lambda_ = torch.tensor(alpha**2 * (n + kappa) - n, dtype=torch.float)

# Compute weights
Wm = torch.full((2 * n + 1,), 1 / (2 * (n + lambda_)))  # Mean weights
Wm[0] = lambda_ / (n + lambda_)

Wc = Wm.clone()  # Covariance weights
Wc[0] += 1 - alpha**2 + beta

# Generate sigma points for the entire batch
scaling = torch.sqrt(n + lambda_)
sigma_points = []

# Central sigma points
sigma_points.append(means)

# Positive and negative directions
for i in range(n):
    sigma_points.append(means + scaling * chol_matrices[:, :, i])
    sigma_points.append(means - scaling * chol_matrices[:, :, i])

# Stack sigma points into a batch tensor
sigma_points = torch.stack(sigma_points, dim=1)  # Shape: (batch, 2n+1, n)
print("sigma points")
print(sigma_points)
# Nonlinear transformation function (example)
# def nonlinear_transform(x):
#     return torch.stack([x[..., 0]**2, torch.sin(x[..., 1])], dim=-1)
def nonlinear_transform(x):
    target = torch.tensor([point_x, point_y])  # Point (3, 4)
    distance = -torch.sqrt(torch.sum((x - target)**2, dim=-1))  # Euclidean distance
    return distance[..., None]  # Add last dimension for compatibility


# Apply the nonlinear transformation to all sigma points
transformed_sigma_points = nonlinear_transform(sigma_points)
print("transform")
print(transformed_sigma_points)
# Compute transformed means (batch)
print(transformed_sigma_points)
transformed_means = torch.sum(Wm[None, :, None] * transformed_sigma_points, dim=1)
print("means")
print(transformed_means.item())
# Compute transformed covariances (batch)
diff = transformed_sigma_points - transformed_means[:, None, :]
transformed_covariances = torch.einsum('bij,bi,bik->bjk', diff, Wc[None], diff)
print("cov")
print(transformed_covariances)
epsilon = torch.tensor(0.95)
gamma = torch.sqrt(epsilon / (1-epsilon))
transformed_means.squeeze().shape, transformed_covariances.squeeze().shape
CVaR = transformed_means.squeeze() + gamma * transformed_covariances.squeeze()
min_idx = CVaR.argmax()


COIV
tensor([[[1.1000, 0.0000],
         [0.0000, 1.6000]]])
L
tensor([[[1.0488, 0.0000],
         [0.0000, 1.2649]]])
sigma points
tensor([[[1.0000, 6.0000],
         [1.0015, 6.0000],
         [0.9985, 6.0000],
         [1.0000, 6.0018],
         [1.0000, 5.9982]]])
transform
tensor([[[-7.2111],
         [-7.2099],
         [-7.2123],
         [-7.2121],
         [-7.2101]]])
tensor([[[-7.2111],
         [-7.2099],
         [-7.2123],
         [-7.2121],
         [-7.2101]]])
means
-7.75
cov
tensor([[[2.2812]]])


In [224]:
Wm

tensor([-986894.0625,  246723.7656,  246723.7656,  246723.7656,  246723.7656])

In [242]:
1 / (2 * (n + lambda_))
(1 / (2*(2 + lambda_))).item()

246723.765625

In [274]:
st = time.time()
point_x, point_y = 30, 40
N = 200
mu_x = torch.linspace(12, 30, N)
mu_y = torch.linspace(40, 60, N)


vertices = [(mu_x.min(), mu_y.min()), (mu_x.min(), mu_y.max()), (mu_x.max(), mu_y.max()), (mu_x.max(), mu_y.min())]

rectangle = Polygon(vertices, closed=True, color='red')

sigma2_x = torch.linspace(0.1, 3, N)
sigma2_y = torch.linspace(0.1, 3, N)

mu_x_grid, mu_y_grid = torch.meshgrid(mu_x, mu_y, indexing="ij")
means = torch.stack([mu_x_grid.flatten(), mu_y_grid.flatten()]).T

sigma2_x_grid, sigma2_y_grid = torch.meshgrid(sigma2_x, sigma2_y, indexing="ij")

pairs = torch.stack([sigma2_x_grid.flatten(), sigma2_y_grid.flatten()])
z = torch.zeros_like(pairs[0,:])
cov = torch.stack([pairs[0,:], z, z, pairs[1,:]], dim=0).reshape(2,2,-1).permute(2,0,1)

chol_matrices = torch.linalg.cholesky(cov)
# cholesky_matrix.shape


# Unscented transformation parameters
alpha = 1e-3  # Scale factor
beta = 2.0    # Optimal for Gaussian distributions
kappa = 0.0   # Secondary scaling parameter
n = means.shape[1]
lambda_ = torch.tensor(alpha**2 * (n + kappa) - n, dtype=torch.float)

# Compute weights
Wm = torch.full((2 * n + 1,), 1 / (2 * (n + lambda_)))  # Mean weights
Wm[0] = lambda_ / (n + lambda_)

Wc = Wm.clone()  # Covariance weights
Wc[0] += 1 - alpha**2 + beta

# Generate sigma points for the entire batch
scaling = torch.sqrt(n + lambda_)
sigma_points = []

# Central sigma points
sigma_points.append(means)

# Positive and negative directions
for i in range(n):
    sigma_points.append(means + scaling * chol_matrices[:, :, i])
    sigma_points.append(means - scaling * chol_matrices[:, :, i])

# Stack sigma points into a batch tensor
sigma_points = torch.stack(sigma_points, dim=1)  # Shape: (batch, 2n+1, n)

# Nonlinear transformation function (example)
# def nonlinear_transform(x):
#     return torch.stack([x[..., 0]**2, torch.sin(x[..., 1])], dim=-1)
def nonlinear_transform(x):
    target = torch.tensor([point_x, point_y])  # Point (3, 4)
    distance = 10-torch.sqrt(torch.sum((x - target)**2, dim=-1))  # Euclidean distance
    return distance  # Add last dimension for compatibility


# Apply the nonlinear transformation to all sigma points
transformed_sigma_points = nonlinear_transform(sigma_points)

# Compute transformed means (batch)
transformed_means = torch.sum(Wm[None, :] * transformed_sigma_points, dim=1)

# Compute transformed covariances (batch)
diff = transformed_sigma_points - transformed_means[:, None]
transformed_variances = torch.sum(Wc[None, :] * diff**2, dim=1)#torch.einsum('bij,bi,bik->bjk', diff, Wc[None], diff)

epsilon = torch.tensor(0.95)
gamma = torch.sqrt(epsilon / (1-epsilon))
transformed_means.squeeze().shape, transformed_covariances.squeeze().shape
CVaR = transformed_means.squeeze() + gamma * transformed_covariances.squeeze()
max_idx = CVaR.argmax()
print('means, cov')
print(means[max_idx], cov[max_idx])
print(time.time() - st)

means, cov
tensor([29.1859, 40.2010]) tensor([[2.8688, 0.0000],
        [0.0000, 0.1291]])
0.004408836364746094


In [276]:
epsilon = 0.95
target_cdf = 0.50
kappa = norm.pdf(norm.ppf(epsilon))/(1-epsilon)
import pickle

with open(f'../1d/a_memory_set/a_memory_{target_cdf}.pickle', 'rb') as fr:
    a_memory = pickle.load(fr)

In [291]:
import time
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

st = time.time()

def plot(point_x,point_y, obs_x, obs_y, obs_sx, obs_sy):
    N = 200
    mu_x = torch.linspace(12, 30, N)
    mu_y = torch.linspace(40, 60, N)
    
    
    vertices = [(mu_x.min(), mu_y.min()), (mu_x.min(), mu_y.max()), (mu_x.max(), mu_y.max()), (mu_x.max(), mu_y.min())]
    
    rectangle = Polygon(vertices, closed=True, color='red')
    
    sigma2_x = torch.linspace(0.1, 3, N)
    sigma2_y = torch.linspace(0.1, 3, N)

    mu_x_grid, mu_y_grid = torch.meshgrid(mu_x, mu_y, indexing="ij")
    means = torch.stack([mu_x_grid.flatten(), mu_y_grid.flatten()]).T

    sigma2_x_grid, sigma2_y_grid = torch.meshgrid(sigma2_x, sigma2_y, indexing="ij")

    pairs = torch.stack([sigma2_x_grid.flatten(), sigma2_y_grid.flatten()])
    z = torch.zeros_like(pairs[0,:])
    cov = torch.stack([pairs[0,:], z, z, pairs[1,:]], dim=0).reshape(2,2,-1).permute(2,0,1)

    chol_matrices = torch.linalg.cholesky(cov)
    # cholesky_matrix.shape


    # Unscented transformation parameters
    alpha = 1e-3  # Scale factor
    beta = 2.0    # Optimal for Gaussian distributions
    kappa = 0.0   # Secondary scaling parameter
    n = means.shape[1]
    lambda_ = torch.tensor(alpha**2 * (n + kappa) - n, dtype=torch.float)

    # Compute weights
    Wm = torch.full((2 * n + 1,), 1 / (2 * (n + lambda_)))  # Mean weights
    Wm[0] = lambda_ / (n + lambda_)

    Wc = Wm.clone()  # Covariance weights
    Wc[0] += 1 - alpha**2 + beta

    # Generate sigma points for the entire batch
    scaling = torch.sqrt(n + lambda_)
    sigma_points = []

    # Central sigma points
    sigma_points.append(means)

    # Positive and negative directions
    for i in range(n):
        sigma_points.append(means + scaling * chol_matrices[:, :, i])
        sigma_points.append(means - scaling * chol_matrices[:, :, i])

    # Stack sigma points into a batch tensor
    sigma_points = torch.stack(sigma_points, dim=1)  # Shape: (batch, 2n+1, n)

    # Nonlinear transformation function (example)
    # def nonlinear_transform(x):
    #     return torch.stack([x[..., 0]**2, torch.sin(x[..., 1])], dim=-1)
    def nonlinear_transform(x):
        target = torch.tensor([point_x, point_y])  # Point (3, 4)
        distance = 10-torch.sqrt(torch.sum((x - target)**2, dim=-1))  # Euclidean distance
        return distance  # Add last dimension for compatibility


    # Apply the nonlinear transformation to all sigma points
    transformed_sigma_points = nonlinear_transform(sigma_points)

    # Compute transformed means (batch)
    transformed_means = torch.sum(Wm[None, :] * transformed_sigma_points, dim=1)

    # Compute transformed covariances (batch)
    diff = transformed_sigma_points - transformed_means[:, None]
    transformed_variances = torch.sum(Wc[None, :] * diff**2, dim=1)#torch.einsum('bij,bi,bik->bjk', diff, Wc[None], diff)

    epsilon = torch.tensor(0.95)
    gamma = torch.sqrt(epsilon / (1-epsilon))
    CVaR = transformed_means.squeeze() + gamma * transformed_covariances.squeeze()
    max_idx = CVaR.argmax()
    print('means, cov')
    print(means[max_idx], cov[max_idx])
    print('max CVaR: ', CVaR.max())

    

    
    
    
    
    
    
    
    fig, ax = plt.subplots(1,2,figsize=(16,8))
    ax[0].add_patch(rectangle)
    
    plot_ellipse(means[max_idx], cov[max_idx], n_std=2, ax=ax[0])
    
    ax[0].scatter(point_x, point_y)
    ax[0].set_xlim(-100, 100)
    ax[0].set_ylim(-100, 100)
    
    ax[1].hist(CVaR, bins=100)
    
    # Calculate New CVaR
    
    mean = torch.tensor([mu_x.min(), mu_y.min()])[None,:]
    cov = torch.tensor([
        [sigma2_x.max(), 0],
        [0, sigma2_y.max()]
    ])[None,:]
    
    mean = torch.tensor([obs_x, obs_y])[None,:]
    cov = torch.tensor([
        [obs_sx, 0],
        [0, obs_sy]
    ])[None,:]
    L = torch.linalg.cholesky(cov)
    
    sigma_points = list()
    sigma_points.append(mean)
    for i in range(n):
        sigma_points.append(mean + scaling * L[:, :, i])
        sigma_points.append(mean - scaling * L[:, :, i])
    sigma_points = torch.stack(sigma_points, dim=1)  # Shape: (batch, 2n+1, n)
    # Apply the nonlinear transformation to all sigma points
    transformed_sigma_points = nonlinear_transform(sigma_points)

    # Compute transformed means (batch)
    transformed_means = torch.sum(Wm[None, :] * transformed_sigma_points, dim=1)

    # Compute transformed covariances (batch)
    diff = transformed_sigma_points - transformed_means[:, None]
    transformed_variances = torch.sum(Wc[None, :] * diff**2, dim=1)#torch.einsum('bij,bi,bik->bjk', diff, Wc[None], diff)
    CVaR = transformed_means.squeeze() + gamma * transformed_covariances.squeeze()
    print("new mean, cov: ",transformed_means.squeeze(), transformed_covariances.squeeze())
    print("new CVaR: ",CVaR)
    ax[1].scatter(CVaR, 0, color='r')
    plot_ellipse(mean[0], cov[0], n_std=2, ax=ax[0], color='green')

    
interact(
        plot,
        point_x=FloatSlider(min=-100, max=100, step=1, value=0, description='Major Axis (a)'),
        point_y=FloatSlider(min=-100, max=100, step=1, value=0, description='Minor Axis (b)'),
        obs_x=FloatSlider(min=12, max=30, step=0.1, value=0, description='Minor Axis (b)'),
        obs_y=FloatSlider(min=40, max=60, step=0.1, value=0, description='Minor Axis (b)'),
        obs_sx=FloatSlider(min=0.1, max=3, step=0.1, value=0, description='Minor Axis (b)'),
        obs_sy=FloatSlider(min=0.1, max=3, step=0.1, value=0, description='Minor Axis (b)'),
);



interactive(children=(FloatSlider(value=0.0, description='Major Axis (a)', min=-100.0, step=1.0), FloatSlider(…

In [33]:
sample_list = list()
for i in range(50):
    sample_list.append([-i, 0])
    
for i in range(50):
    sample_list.append([-50, i])
    
for i in range(100):
    sample_list.append([i-50, 50])
    
for i in range(100):
    sample_list.append([50, 50-i])
    
for i in range(150):
    sample_list.append([50-i, -50])
    
for i in range(150):
    sample_list.append([-100, i-50])
    
for i in range(200):
    sample_list.append([i-100, 100])
    
for i in range(200):
    sample_list.append([100, i-100])

In [34]:
from matplotlib.patches import Polygon
from ipywidgets import interact, FloatSlider
from tqdm import tqdm

for idx, (x,y) in tqdm(enumerate(sample_list)):
    fig, ax = plt.subplots(figsize=(8,8))
    
    sample = [x, y]
    input_data = torch.tensor(cart2polar(*sample), dtype=torch.float)[None,:]
    with torch.no_grad():
        pred = model(input_data)
    x_params = [pred[0][0,0], pred[1][0,0], pred[2][0,0], pred[3][0,0]]
    y_params = [pred[0][0,1], pred[1][0,1], pred[2][0,1], pred[3][0,1]]

    
    # ambiguity set
    ## x axis
    mu_0, lambda_, alpha, beta = x_params
    try:
        zx, zy = a_memory[float(f'{alpha:.02f}')]
    except:
        zx, zy = a_memory[float(f'{1.01}')]
    delta = zx / np.sqrt(lambda_/beta)
    x_mu_low, x_mu_high = mu_0 - delta, mu_0 + delta
    
    ## y axis
    mu_0, lambda_, alpha, beta = y_params
    try:
        zx, zy = a_memory[float(f'{alpha:.02f}')]
    except:
        zx, zy = a_memory[float(f'{1.01}')]
    delta = zx / np.sqrt(lambda_/beta)
    y_mu_low, y_mu_high = mu_0 - delta, mu_0 + delta
    
    vertices = [(x_mu_low, y_mu_low), (x_mu_low, y_mu_high), (x_mu_high, y_mu_high), (x_mu_high, y_mu_low)]
    
    rectangle = Polygon(vertices, closed=True, color='red')
    # sampling
    for _ in range(500):
        x_dist_params = NIG_sample(x_params)
        y_dist_params = NIG_sample(y_params)

        mean = np.array([x_dist_params[0], y_dist_params[0]])
        cov = np.array([
            [x_dist_params[1], 0],
            [0, y_dist_params[1]]
        ])


        # 공분산 행렬의 고유값과 고유벡터 계산
        eigvals, eigvecs = np.linalg.eigh(cov)

        # 1-sigma 수준에서 타원의 축 반지름 계산
        axis_lengths = np.sqrt(eigvals)

        # 타원의 각도 계산 (라디안)
        angle = np.arctan2(eigvecs[1, 0], eigvecs[0, 0])

        # 타원 좌표 생성
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse = np.array([axis_lengths[0] * np.cos(theta), axis_lengths[1] * np.sin(theta)])

        # 타원의 회전 적용
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                                     [np.sin(angle), np.cos(angle)]])
        rotated_ellipse = rotation_matrix @ ellipse

        # 타원을 평균 좌표로 이동
        ellipse_x, ellipse_y = rotated_ellipse[0] + mean[0], rotated_ellipse[1] + mean[1]
        if (x_mu_low < mean[0] and mean[0] < x_mu_high) and (y_mu_low < mean[1] and mean[1] < y_mu_high):
            color = 'lime'
            alpha = 0.1
            label = 'in ambiguity set'
        else:
            color = 'skyblue'
            alpha = 0.8
            label = 'out of ambiguity set'
        plt.plot(ellipse_x, ellipse_y, color=color, alpha=alpha, label=label)
    ax.add_patch(rectangle)

    plt.scatter(*sample, color='g', label='Ground Truth')
    plt.xlim(-100, 100)
    plt.ylim(-100, 100)
    handles, labels = plt.gca().get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))  # 중복 제거

    # 고유한 레이블만 포함하는 legend 표시
    plt.legend(unique_labels.values(), unique_labels.keys())   
    plt.savefig(f'record/{idx}.png')
    plt.close()


1000it [15:46,  1.06it/s]
